In [ ]:
## Mode 1: Automatic AI vs. Random Word

# This mode simulates the AI playing against a randomly chosen target word from `words.txt`. It uses the `test_highestFrequency` function from `main.py` (which we import as `gameEngine`).

# Change the number inside `test_highestFrequency()` to run more simulations (e.g., `test_highestFrequency(10)`).

In [ ]:
import main as gameEngine
import random

gameEngine.available_words = [i[:-1] for i in open("words.txt", "r").readlines()]
gameEngine.permanent_answers = [i[:-1] for i in open("words.txt", "r").readlines()]
gameEngine.wordsAllowed = [i[:-1] for i in open("wordsAllowed.txt", "r").readlines()]

print("Running 1 automatic game simulation...")
gameEngine.test_highestFrequency(1)

print("\nSimulation complete.")

In [ ]:
# ## Mode 2: AI vs. User-Defined Word

# In this mode, you provide a specific 5-letter target word (which must be a valid potential answer from `words.txt`), and the AI will attempt to guess it.

In [1]:
import main as gameEngine
import random
import math

def solve_specific_word(target_word):
    target_word = target_word.lower()

    if len(target_word) != 5:
        print(f"Error: '{target_word}' is not 5 letters long.")
        return

    gameEngine.available_words = [i[:-1] for i in open("words.txt", "r").readlines()]
    gameEngine.permanent_answers = [i[:-1] for i in open("words.txt", "r").readlines()]
    gameEngine.wordsAllowed = [i[:-1] for i in open("wordsAllowed.txt", "r").readlines()]
    initial_word_list = gameEngine.permanent_answers[:]

    if target_word not in initial_word_list:
        print(f"Error: '{target_word}' is not in the list of possible answers (words.txt).")
        return

    # --- Simulation Logic ---
    available_words = initial_word_list[:]
    steps = 0
    guess_history = []

    print(f"Target Word: {target_word}")

    # --- First Guess ---
    guess = "salet"
    steps = 1
    guess_history.append(guess)
    colors = gameEngine.get_guess_colors(guess, target_word) 
    emoji_output = gameEngine.format_colors_to_emoji(colors)
    print(f"Guess {steps}: {guess} -> {emoji_output}")

    if guess == target_word:
        print(f"Solved in {steps} steps!")
        print(f"History: {guess_history}")
        return

    available_words = gameEngine.filter_words(available_words, guess, target_word)
    print(f"  Remaining possible words: {len(available_words)}")
    if target_word not in available_words and len(available_words) > 0:
         print(f"Warning: Target word '{target_word}' was filtered out! List: {available_words[:10]}")

    # --- Subsequent Guesses (Loop) ---
    for j in range(5):
        if not available_words:
             print("Error: No possible words left in list.")
             steps = -1 # Indicate failure state
             break

        if len(available_words) == 1:
            guess = available_words[0]
        elif gameEngine.isBlimp(available_words):
            print("  (Blimp condition detected)")
            guess = gameEngine.blimpSearch(available_words)
        else:
            guess = gameEngine.getMaxValue1(available_words)

        steps += 1
        guess_history.append(guess)
        colors = gameEngine.get_guess_colors(guess, target_word) 
        emoji_output = gameEngine.format_colors_to_emoji(colors)
        print(f"Guess {steps}: {guess} -> {emoji_output}")

        if guess == target_word:
            print(f"Solved in {steps} steps!")
            print(f"History: {guess_history}")
            return

        available_words = gameEngine.filter_words(available_words, guess, target_word)
        print(f"  Remaining possible words: {len(available_words)}")
        if target_word not in available_words and len(available_words) > 0:
            print(f"Warning: Target word '{target_word}' was filtered out! List: {available_words[:10]}")

    if steps != -1 and guess != target_word:
        print(f"Failed to solve in 6 steps.")
        print(f"History: {guess_history}")
        print(f"Remaining possibilities: {available_words}")

user_target = input("Enter the 5-letter target word (must be in words.txt): ")
solve_specific_word(user_target)

Enter the 5-letter target word (must be in words.txt):  steal


Target Word: steal
Guess 1: salet -> 🟩🟨🟨🟨🟨
  Remaining possible words: 3
  (Blimp condition detected)
Guess 2: slate -> 🟩🟨🟨🟨🟨
  Remaining possible words: 1
Guess 3: steal -> 🟩🟩🟩🟩🟩
Solved in 3 steps!
History: ['salet', 'slate', 'steal']


In [ ]:
## Mode 3: Human VS AI Wordle

In [1]:
import main as gameEngine
import random
import math

def play_human_vs_ai():
    """Plays a game of Wordle where a human competes against the AI, with difficulty setting."""

    while True:
        difficulty = input("Choose difficulty (easy/hard): ").lower().strip()
        if difficulty in ['easy', 'hard']:
            break
        else:
            print("Invalid input. Please type 'easy' or 'hard'.")

    # --- Setup ---
    try:
        current_permanent_answers = [i[:-1] for i in open("words.txt", "r").readlines()]
        current_words_allowed = [i[:-1] for i in open("wordsAllowed.txt", "r").readlines()]
        gameEngine.permanent_answers = current_permanent_answers[:]
        gameEngine.wordsAllowed = current_words_allowed[:]
        gameEngine.available_words = current_permanent_answers[:]
    except FileNotFoundError:
        print("Error: words.txt or wordsAllowed.txt not found in the current directory.")
        return

    if not current_permanent_answers:
        print("Error: words.txt is empty or could not be read.")
        return

    target_word = random.choice(current_permanent_answers)
    print("\n--- Human vs AI Wordle ---")
    print(f"Difficulty: {difficulty.capitalize()}")
    # print(f"(DEBUG: The word is {target_word})")

    human_guesses_history = []
    ai_guesses_history = []
    ai_available_words = current_permanent_answers[:]

    # --- Game Loop ---
    for turn in range(1, 7):
        print(f"\n--- Turn {turn} ---")

        while True:
            human_guess = input(f"Your guess ({turn}/6): ").lower()
            if len(human_guess) != 5:
                print("Guess must be 5 letters long.")
            elif human_guess not in current_words_allowed and human_guess not in current_permanent_answers:
                 print(f"'{human_guess}' is not in the list of allowed words.")
            else:
                break

        human_colors = gameEngine.get_guess_colors(human_guess, target_word)
        human_emoji = gameEngine.format_colors_to_emoji(human_colors)
        human_guesses_history.append(f"{human_guess} -> {human_emoji}")
        print(f"Your result: {human_emoji}")

        if human_guess == target_word:
            print(f"\nCongratulations! You guessed the word '{target_word}' in {turn} turns!")
            print("Human wins! 🎉")
            return

        print("AI is thinking...")
        if turn == 1:
            ai_guess = "salet"
        else:
            if ai_guesses_history:
                 last_ai_guess = ai_guesses_history[-1].split(" ")[0]
                 ai_available_words = gameEngine.filter_words(ai_available_words[:], last_ai_guess, target_word)
                 # print(f"(AI Debug: Remaining words: {len(ai_available_words)})")

            if not ai_available_words:
                 print("AI Error: No possible words left for AI!")
                 ai_guess = "error"
            elif len(ai_available_words) == 1:
                 ai_guess = ai_available_words[0]
            elif gameEngine.isBlimp(ai_available_words):
                 print("(AI detected blimp condition)")
                 ai_guess = gameEngine.blimpSearch(ai_available_words)
            else:
                 ai_guess = gameEngine.getMaxValue1(ai_available_words)

        ai_colors = gameEngine.get_guess_colors(ai_guess, target_word)
        ai_emoji = gameEngine.format_colors_to_emoji(ai_colors)

        ai_guesses_history.append(f"{ai_guess} -> {ai_emoji}")

        if difficulty == 'easy':
            print(f"AI guess ({turn}/6): {ai_guess} -> {ai_emoji}")
        else:
            print(f"AI guess ({turn}/6): ????? -> {ai_emoji}") # Hide the word

        if ai_guess == target_word:
            print(f"\nThe AI guessed the word '{target_word}' in {turn} turns!")
            print("AI wins! 🤖")
            return 

    # --- No winner after 6 turns ---
    print("\n--- Game Over ---")
    print(f"Neither you nor the AI guessed the word in 6 turns.")
    print(f"The word was: {target_word}")
    print("\nYour Guesses:")
    for guess_info in human_guesses_history:
        print(guess_info)
    print("\nAI Guesses:")
    for guess_info in ai_guesses_history:
        print(guess_info)

play_human_vs_ai()

Choose difficulty (easy/hard):  easy



--- Human vs AI Wordle ---
Difficulty: Easy

--- Turn 1 ---


Your guess (1/6):  tests


Your result: ⬛⬛⬛⬛⬛
AI is thinking...
AI guess (1/6): salet -> ⬛⬛🟨⬛⬛

--- Turn 2 ---


Your guess (2/6):  latte


...

Your result: 🟨⬛⬛⬛⬛
AI is thinking...
(AI detected blimp condition)
AI guess (3/6): bumfs -> ⬛🟨⬛⬛⬛

--- Turn 4 ---


Your guess (4/6):  fumbs


'fumbs' is not in the list of allowed words.


Your guess (4/6):  house


Your result: ⬛⬛🟩⬛⬛
AI is thinking...
AI guess (4/6): plunk -> 🟩🟩🟩🟩🟩

The AI guessed the word 'plunk' in 4 turns!
AI wins! 🤖


In [ ]:
## Mode 4: Wordle Helper AI

In [ ]:
import main as gameEngine
import random
import math
import re

def play_ai_helper_mode():
    """Plays Wordle with AI assistance."""

    try:
        current_permanent_answers = [i[:-1] for i in open("words.txt", "r").readlines()]
        current_words_allowed = [i[:-1] for i in open("wordsAllowed.txt", "r").readlines()]
        gameEngine.permanent_answers = current_permanent_answers[:]
        gameEngine.wordsAllowed = current_words_allowed[:]
    except FileNotFoundError:
        print("Error: words.txt or wordsAllowed.txt not found in the current directory.")
        return

    if not current_permanent_answers:
        print("Error: words.txt is empty or could not be read.")
        return

    ai_available_words = current_permanent_answers[:]
    guess_history = []

    print("\n--- Wordle AI Helper ---")
    print("Instructions:")
    print("1. AI suggests a word.")
    print("2. Enter that word into your Wordle game.")
    print("3. Enter the 5-letter color result back here.")
    print("   Use 'B' for Black/Gray, 'Y' for Yellow, 'G' for Green (e.g., 'BGYBB').")

    # --- Game Loop ---
    for turn in range(1, 7):
        print(f"\n--- Turn {turn} ---")

        if turn == 1:
            ai_guess = "salet"
        else:
            if not ai_available_words:
                print("Error: No possible words left based on feedback!")
                return
            elif len(ai_available_words) == 1:
                ai_guess = ai_available_words[0] 
            elif gameEngine.isBlimp(ai_available_words):
                print("(AI detected blimp condition, choosing differentiating word...)")
                ai_guess = gameEngine.blimpSearch(ai_available_words)
            else:
                ai_guess = gameEngine.getMaxValue1(ai_available_words) 

        print(f"AI suggests guessing: {ai_guess.upper()}")

        color_feedback_byg = ""
        color_feedback_numeric = ""
        while True:
            color_feedback_byg = input("Enter the 5-letter color result (B/Y/G): ").upper().strip()
            if len(color_feedback_byg) == 5 and re.match("^[BGY]{5}$", color_feedback_byg):
                 color_feedback_numeric = color_feedback_byg.replace('B', '0').replace('Y', '1').replace('G', '2')
                 break
            else:
                 print("Invalid input. Please enter exactly 5 letters using only B, Y, or G.")

        emoji_feedback = gameEngine.format_colors_to_emoji(color_feedback_byg)
        guess_history.append(f"{ai_guess} -> {emoji_feedback}")

        if color_feedback_byg == "GGGGG":
            print(f"\nCongratulations! You found the word in {turn} turns!")
            print("History:")
            for entry in guess_history:
                print(entry)
            return # End game

        # --- AI Filters List  ---
        ai_available_words = gameEngine.gameFilter(ai_guess, color_feedback_numeric, ai_available_words)
        remaining_count = len(ai_available_words)

        print(f"Possible words remaining: {remaining_count}")
        if remaining_count <= 10 and remaining_count > 0:
             print(f"Possibilities: {', '.join(ai_available_words)}")
        elif remaining_count == 0:
             print("Uh oh! No words match the feedback provided. Double-check your color inputs.")
             return


    # --- End of Game  ---
    print("\n--- Game Over ---")
    print("You've used all 6 turns.")
    if len(ai_available_words) == 1:
        print(f"The only remaining possibility was: {ai_available_words[0]}")
    elif len(ai_available_words) > 1 and len(ai_available_words) <= 10:
        print(f"The remaining possibilities were: {', '.join(ai_available_words)}")
    elif len(ai_available_words) > 10:
         print(f"There were still {len(ai_available_words)} possibilities left.")

    print("\nHistory:")
    for entry in guess_history:
        print(entry)
        
play_ai_helper_mode()


--- Wordle AI Helper ---
Instructions:
1. AI suggests a word.
2. Enter that word into your Wordle game.
3. Enter the 5-letter color result back here.
   Use 'B' for Black/Gray, 'Y' for Yellow, 'G' for Green (e.g., 'BGYBB').

--- Turn 1 ---
AI suggests guessing: SALET


Enter the 5-letter color result (B/Y/G):  BBBBB


Possible words remaining: 221

--- Turn 2 ---
AI suggests guessing: IRONY


Enter the 5-letter color result (B/Y/G):  bbybg


Possible words remaining: 8
Possibilities: bobby, comfy, dodgy, dowdy, foggy, hobby, howdy, poppy

--- Turn 3 ---
(AI detected blimp condition, choosing differentiating word...)
AI suggests guessing: BEDIM


Enter the 5-letter color result (B/Y/G):  bbbbb


Possible words remaining: 2
Possibilities: foggy, poppy

--- Turn 4 ---
(AI detected blimp condition, choosing differentiating word...)
AI suggests guessing: FOGGY


## Full Simulation & Histogram

This mode runs the solver against **every single word** in `words.txt` to generate a complete performance profile.

It then plots a histogram showing the distribution of guesses required (1, 2, 3, 4, 5, 6, or DNF for failures).

**Warning:** This will take some time to run! There are 2,315 words in `words.txt`. Depending on your computer, expect it to take a few minutes.

In [ ]:
import main as gameEngine
import matplotlib.pyplot as plt
import numpy as np
import time
import math

%matplotlib inline

def solve_specific_word_for_stats(target_word, game_engine, initial_word_list):
    """Solves for a target word and returns the number of steps (7 for DNF)."""
    available_words = initial_word_list[:]
    steps = 0

    guess = "salet"
    steps = 1
    if guess == target_word:
        return steps
    available_words = game_engine.filter_words(available_words, guess, target_word)
    if target_word not in available_words and len(available_words) > 0:
         # This check is for debugging the filter_words logic, if needed
         # print(f"Warning: Target {target_word} filtered out by {guess}")
         pass

    for j in range(5):
        if not available_words:
             return 7

        if len(available_words) == 1:
            guess = available_words[0]
        elif game_engine.isBlimp(available_words):
            # print(f"Blimp detected for {target_word} with list: {available_words}")
            guess = game_engine.blimpSearch(available_words)
        else:
            guess = game_engine.getMaxValue1(available_words)

        steps += 1
        if guess == target_word:
            return steps

        available_words = game_engine.filter_words(available_words, guess, target_word)
        if target_word not in available_words and len(available_words) > 0:
            # print(f"Warning: Target {target_word} filtered out by {guess}")
            pass

    return 7


# --- Main Execution ---
print("Starting full simulation for all words in words.txt...")
print("This may take several minutes.")

permanent_answers = [i[:-1] for i in open("words.txt", "r").readlines()]
gameEngine.available_words = permanent_answers[:] 
gameEngine.permanent_answers = permanent_answers[:]
gameEngine.wordsAllowed = [i[:-1] for i in open("wordsAllowed.txt", "r").readlines()]

results = []
dnf_words = [] # <- List to store failed words
start_time = time.time()
total_words = len(permanent_answers)

for i, word in enumerate(permanent_answers):
    if (i+1) % 200 == 0:
        print(f"... processed {i+1}/{total_words} words ...")
    
    steps = solve_specific_word_for_stats(word, gameEngine, permanent_answers)
    results.append(steps)
    
    if steps == 7:
        dnf_words.append(word)
            
end_time = time.time()
print(f"\nSimulation complete. Processed {total_words} words in {end_time - start_time:.2f} seconds.")
print("Generating histogram...")

results_array = np.array(results)

success_games = results_array[results_array <= 6]
dnf_count = np.count_nonzero(results_array == 7)
total_games = len(results_array)

if total_games > 0:
    success_rate = len(success_games) / total_games
    avg_steps = np.mean(success_games) if len(success_games) > 0 else 0
    
    print(f"--- Overall Stats ---")
    print(f"Total Games: {total_games}")
    print(f"Success Rate: {success_rate * 100:.2f}%")
    print(f"Failed Games (DNF): {dnf_count}")
    print(f"Average Steps (on success): {avg_steps:.4f}")
    if dnf_count > 0: #
        print(f"Failed Words: {', '.join(dnf_words)}")
    print("\nDistribution of Guesses:")
    for i in range(1, 8):
        count = np.count_nonzero(results_array == i)
        label = f"DNF (7)" if i == 7 else f"{i} Steps"
        print(f"  {label}: {count} games")

# Create the histogram
bins = np.arange(1, 9)

plt.figure(figsize=(10, 6))
plt.hist(results_array, bins=bins, align='left', edgecolor='black', rwidth=0.8, color='skyblue')

plt.title('Wordle Solver Performance (Highest Frequency Strategy)')
plt.xlabel('Steps to Solve')
plt.ylabel('Number of Games')

tick_labels = [str(i) for i in range(1, 7)] + ['DNF (7+)']
plt.xticks(ticks=np.arange(1, 8), labels=tick_labels)

plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
